## Task 2: Words as Features

### 2.1 Extracting tokens/words and their frequencies

In [1]:
import gzip
import json
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

with gzip.open("dataset/goemotions.json.gz", "rb") as f:
    data = json.loads(f.read())

dataset = np.array(data)

# Extracting posts, emotions and sentiments
posts = dataset[:, 0]
emotions = dataset[:, 1]
sentiments = dataset[:, 2]

vectorizer = CountVectorizer()
# Creates a document-term matrix
posts_features = vectorizer.fit_transform(posts)

# Printing the number of tokens (size of the vocabulary)
print("Number of unique tokens (size of vocabulary) in the dataset:", len(vectorizer.get_feature_names_out()))

Number of unique tokens (size of vocabulary) in the dataset: 30449


### 2.2 Split the dataset to 80% for training and 20% for testing for each model

In [2]:
# Training and testing dataset for posts and emotions
x_train_emo, x_test_emo, y_train_emo, y_test_emo = train_test_split(posts_features, emotions, test_size=0.2, random_state=0)

# Training and testing dataset for posts and sentiments
x_train_sen, x_test_sen, y_train_sen, y_test_sen = train_test_split(posts_features, sentiments, test_size=0.2, random_state=0)